Generating compositions from atomic parameters using brute force search.

In [356]:
from datetime import date
from datetime import datetime
import time
import os
import warnings
from tqdm import tqdm

import numpy as np
import pandas as pd
import random
from scipy.stats import norm
import math
from itertools import product
from fractions import Fraction

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

from pymatgen.core import Species
from pymatgen.core.periodic_table import Element

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

In [357]:
# Create list of discrete atomic fractions

#a_f = [0, 1/9, 1/8, 1/7, 1/6, 1/5, 1/4, 1/3, 1/2, 2/3, 3/4, 1]

# Create constraints
num_A_lim = 5 # upper limit for Num A
num_B_lim = 1 # upper limit for Num B

iter_i = 500000
tol = 0.1
best_distance = 9999.999

In [358]:
#Set save directory
results_dir = "Atomic_params_to_compositions"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [359]:
# Enter desired atomic parameters

RA_best = 1.0911534534935248
RB_best = 0.7199999999999999
MA_best = 172.63460807159552
MB_best = 91.224
Entropy_config_best = 0.8238972178945182

Opt_params = {'RA_best':RA_best, 'RB_best':RB_best, 'MA_best':MA_best, 'MB_best':MB_best, 'Entropy_best':Entropy_config_best}

In [360]:
# Set species constraints
LaA_c = 1
NdA_c = 1
SmA_c = 1
GdA_c = 1
EuA_c = 1
TmA_c = 1
YbA_c = 1
YA_c = 1
DyA_c = 1
HoA_c = 1
ErA_c = 1
PrA_c = 1
CeA_c = 1
ZrB_c = 1
HfB_c = 1
SnB_c = 1
CeB_c = 1
TiB_c = 1

Chem_sp_C = {'LaA':LaA_c, 'NdA':NdA_c, 'SmA':SmA_c, 'GdA':GdA_c, 'EuA':EuA_c, 'TmA':TmA_c, 'YbA':YbA_c, 'YA':YA_c, \
             'DyA':DyA_c, 'HoA':HoA_c, 'ErA':ErA_c, 'PrA':PrA_c, 'CeA':CeA_c, 'ZrB':ZrB_c, 'HfB':HfB_c, 'SnB':SnB_c,\
             'CeB':CeB_c, 'TiB':TiB_c}

In [361]:
#Set bounds for normalizing data (Output only)


# Fixing RA range between radius of La and Yb
RA_bound = [0.985, 1.16]

# RB min/max set to radii of Ti and Ce, respectively
RB_bound = [0.605, 0.87]

# Fixing RA/RB range from Yb to upper limit for stable pyrohclore phase
RA_RB_bound = [1.368, 1.78]

# Fixing MA range between mass of Y and Yb
MA_bound = [88.90585, 173.04]

# Fixing MB range to min/max of Wright et al.
MB_bound= [47.867, 134.857]

# 0 for Flourite and 1 for Pyrochlore phase
P_bound = [0, 1]

# Lattice parameter min from Wright et al. and max set to value for La2Zr2O7 from Wan et al. as it is higher
a_2_bound = [10.221114, 10.81]

# Min density of La2Zr2O7 and max from Wright et al.
Density_bound = [6.05, 7.795366]

# Max density set slightly above the density for equimolar, 9 element compositions (2.6)
Entropy_bound = [0, 3]

# Thermal Conductivity range set between min/max of Wright et al.
TC_bound = [1.36, 2.88]

min_values = [RA_bound[0], RB_bound[0], MA_bound[0], MB_bound[0], Entropy_bound[0]]

max_values = [RA_bound[1], RB_bound[1], MA_bound[1], MB_bound[1], Entropy_bound[1]]

normalization_vals = {'RA':(RA_bound[0], RA_bound[1]), 'RB':(RB_bound[0], RB_bound[1]), \
                      'MA':(MA_bound[0], MA_bound[1]), 'MB':(MB_bound[0], MB_bound[1]), \
                      'Entropy_conf': (Entropy_bound[0], Entropy_bound[1])}

In [362]:
#Import and check data
data_file = r'C:\Users\ezxac5\Documents\GitHub\Materials-Discovery\Machine_learning\Active_learning\Pyrochlore_database_atomic_param_unnormalized.csv'
data = pd.read_csv(data_file)                            #Import CSV Data

In [363]:
temp_columns = data.columns
temp_columns = temp_columns[1:]
data.drop_duplicates(subset=temp_columns) # Drop duplicate data

,Comp,LaA,NdA,SmA,GdA,EuA,TmA,YbA,YA,DyA,...,RA,RB,RA/RB,MA,MB,P,a2,rho,Entropy_conf,Distance
0,La2Zr2O7,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.00,0.00,...,1.160000,0.720000,1.611111,138.905470,91.224000,1,10.728332,6.135069,0.171259,2.916340
1,(La0.2Nd0.2Sm0.2Eu0.2Gd0.2)2Zr2O7,0.200000,0.200000,0.200000,0.200000,0.200000,0.0,0.000,0.00,0.00,...,1.093400,0.720000,1.518611,148.544294,91.224000,1,10.602573,6.570843,1.634384,9.072623
2,(La0.2Nd0.2Sm0.2Gd0.2Yb0.2)2Zr2O7,0.200000,0.200000,0.200000,0.200000,0.000000,0.0,0.200,0.00,0.00,...,1.077200,0.720000,1.496111,152.759494,91.224000,1,10.570717,6.725246,1.634384,13.254081
3,(La0.2Nd0.2Y0.2Er0.2Yb0.2)2Zr2O7,0.200000,0.200000,0.000000,0.000000,0.000000,0.0,0.200,0.20,0.00,...,1.055400,0.720000,1.465833,142.470464,91.224000,1,10.527066,6.574921,1.634384,3.204955
4,(Dy0.2 Y0.2Ho0.2Er0.2Yb0.2)2Zr2O7,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200,0.20,0.20,...,1.010000,0.720000,1.402778,151.327034,91.224000,0,10.433279,6.961026,1.634384,11.854600
5,Yb2Zr2O7,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000,0.00,0.00,...,0.985000,0.720000,1.368056,173.040000,91.224000,0,10.379972,7.584658,0.171259,33.590813
6,Y2Zr2O7,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,1.00,0.00,...,1.019000,0.720000,1.415278,88.905850,91.224000,0,10.452180,5.470916,0.171259,50.756515
8,Sm 2 Zr 2 O 7,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000,0.00,0.00,...,1.079000,0.720000,1.498611,150.360000,91.224000,1,10.574281,6.664528,0.171259,11.148703
9,La 2 (Hf 1/2 Zr 1/2 ) 2 O 7,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.00,0.00,...,1.160000,0.715000,1.622378,138.905470,134.857000,1,10.713793,7.102760,0.801393,2.311317
10,Sm 2 (Sn 1/4 Ti 1/4 Hf 1/4 Zr 1/4 ) 2 O 7,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000,0.00,0.00,...,1.079000,0.681250,1.583853,150.360000,109.072750,1,10.466946,7.285213,1.431527,10.897798


In [364]:
data_input = data.drop(columns=['Comp', 'RA', 'RB', 'RA/RB', 'MA', 'MB', 'P', 'a2', 'rho', 'Entropy_conf', 'Distance'])      # Specify input descriptors
target = data[['RA', 'RB', 'MA', 'MB', 'Entropy_conf']]                                                               # Specify output descriptors        

In [365]:
# Box plot to identify outliers
rows_c = 3
cols_c = 6
fig = make_subplots(rows = rows_c, cols = cols_c)
row_no = col_no = 1

for col in data_input.columns:
    fig.add_trace(
    go.Box(y=data[col], name = col),
    row = row_no,
    col = col_no,
    )
    col_no += 1
    if col_no % (cols_c+1) == 0:
        row_no += 1
        col_no = 1
        
fig.update_layout(title_text = "Chemical species outlier detection through boxplot")
fig.show()
fig.write_html(results_dir + '\\' + f"Chem_spcs_outlier_detection.html")

In [366]:
# Box plot to identify outliers
rows_c = 2
cols_c = 3
fig = make_subplots(rows = rows_c, cols = cols_c)
row_no = col_no = 1

for col in target.columns:
    fig.add_trace(
    go.Box(y=target[col], name = col),
    row = row_no,
    col = col_no,
    )
    col_no += 1
    if col_no % (cols_c+1) == 0:
        row_no += 1
        col_no = 1
        
fig.update_layout(title_text = "Crystal parameters outlier detection through boxplot")
fig.show()
fig.write_html(results_dir + '\\' + f"Crys_params_outlier_detection.html")

In [367]:
#Set bounds for normalizing data (Output only)


# Fixing RA range between radius of La and Yb
RA_bound = [0.985, 1.16]

# RB min/max set to radii of Ti and Ce, respectively
RB_bound = [0.605, 0.87]

# Fixing RA/RB range from Yb to upper limit for stable pyrohclore phase
RA_RB_bound = [1.368, 1.78]

# Fixing MA range between mass of Y and Yb
MA_bound = [88.90585, 173.04]

# Fixing MB range to min/max of Wright et al.
MB_bound= [47.867, 134.857]

# 0 for Flourite and 1 for Pyrochlore phase
P_bound = [0, 1]

# Lattice parameter min from Wright et al. and max set to value for La2Zr2O7 from Wan et al. as it is higher
a_2_bound = [10.221114, 10.81]

# Min density of La2Zr2O7 and max from Wright et al.
Density_bound = [6.05, 7.795366]

# Max density set slightly above the density for equimolar, 9 element compositions (2.6)
Entropy_bound = [0, 3]

# Thermal Conductivity range set between min/max of Wright et al.
TC_bound = [1.36, 2.88]

min_values = [RA_bound[0], RB_bound[0], MA_bound[0], MB_bound[0], Entropy_bound[0]]

max_values = [RA_bound[1], RB_bound[1], MA_bound[1], MB_bound[1], Entropy_bound[1]]

normalization_vals = {'RA':(RA_bound[0], RA_bound[1]), 'RB':(RB_bound[0], RB_bound[1]), \
                      'MA':(MA_bound[0], MA_bound[1]), 'MB':(MB_bound[0], MB_bound[1]), \
                      'Entropy_conf': (Entropy_bound[0], Entropy_bound[1])}

In [368]:
# Normalise target variables (input (composition) is already normalised)

target_array = np.array(target)

scaler = MinMaxScaler()

scaler.fit([min_values, max_values])

target_norm = scaler.transform(target_array)

In [369]:
input_array = np.array(data_input) # Convert input into numpy array

In [370]:
def unnormalize(value, min_value, max_value):
        return ((value*(max_value - min_value)) + min_value)

In [371]:
columns = target.columns # Get column names

In [372]:
def get_atomic_property(element, oxidation_state, coordination):
    '''
    Get shannon ionic radii for each element.
    Oxidation state is 3 for A cations and 4 for B cations.
    Coordination is str(VIII) for A cations and str(VI) for B cations.
    '''
    try:
        # Create a Species object with given element and oxidation state
        specie = Species(element, oxidation_state)
        
        # Retrieve Shannon ionic radii for the specified coordination
        shannon_radii = specie.get_shannon_radius(cn=coordination)
        atom_mass = specie.atomic_mass
        
        atom_prop = [shannon_radii, atom_mass]
        
        return atom_prop
        
    except:
        print(f'No ionic radius found for {coordination} coordination')

In [373]:
def abstract_distance(target_params, predicted_parameters):
    '''
    Convert combination of atomic parameters into euclidean distance
    '''
    
    D = 0
    for i in range(len(predicted_parameters[0])):
        D += (predicted_parameters[0][i] - target_params[0][i])**2
        
    D = math.sqrt(D)
    return(D)

In [374]:
def gen_comp(num_A_lim, num_B_lim, species_A, species_B):
    
    '''
    Generate compositions with constraints
    
    num_A_lim: upper limit of number of elements in A
    num_B_lim: upper limit of number of elements in B
    species_A: list of species in A sub-lattice
    species_B: list of species in B sub-lattice
    '''

    num_A = random.randint(1, num_A_lim) # Randomly generate number of elements in A upto num_A_lim
    num_B = random.randint(1, num_B_lim) # Randomly generate number of elements in B upto num_B_lim
    
    species_index_A = random.sample(range(len(species_A)), num_A)
    species_index_B = random.sample(range(len(species_B)), num_B)

    new_composition = {}
    
    comp_species_A = []

    for i in species_index_A:
        comp_species_A.append(species_A[i])


    # Randomly select num_A numbers from the list
    a_f_A = np.random.choice(range(1,10), num_A, replace=True)

    # Calculate the sum of the selected numbers
    total_sum_A = np.sum(a_f_A)
        
    count=0
    for species in comp_species_A:
        new_composition[species] = a_f_A[count]/total_sum_A
        count+=1
        
    # Repeat steps for species in B sub-lattice
    comp_species_B = []
    
    for i in species_index_B:
        comp_species_B.append(species_B[i])

    a_f_B = np.random.choice(range(1,10), num_B, replace=True)
    
    total_sum_B = np.sum(a_f_B)
    
    count=0
    for species in comp_species_B:
        new_composition[species] = a_f_B[count]/total_sum_B
        count+=1

    val = [new_composition, num_A, num_B]
    
    return(val)

In [375]:
def train_model(columns, target_n, model, input_array, target_norm, random_state=42):
    
    '''
    Train model
    Evaluate with k_fold
    Print error metrics and plot predicted vs actual values
    (columns: pandas indexer containing column names, target_n: name of target variable, model: machine learning model,
    input_array: numpy array containing input variables, target_norm: numpy array containing target variables, 
    random_state: random number state) 
    '''
    
    model = model
    
    # Get column index for target variable
    target_index = columns.get_indexer([target_n])
    target_index=target_index[0]
    print(f'Target: {target_n}')
    print(target_index)
    

    Input_train = input_array
    Output_train = target_norm[:, target_index]

    # Train model
    model.fit(Input_train, Output_train)
        

    
    return(model)

In [376]:
data_species = data_input.columns

In [377]:
# list of species that fit within constraints
data_species_A_c = []
data_species_B_c = []

for key, val in Chem_sp_C.items():
    if val != 0:
        if key[-1]=='A':
            data_species_A_c.append(key)
        else:
            data_species_B_c.append(key)

In [378]:
print(data_species_A_c)
print(len(data_species_A_c))
print(data_species_B_c)

['LaA', 'NdA', 'SmA', 'GdA', 'EuA', 'TmA', 'YbA', 'YA', 'DyA', 'HoA', 'ErA', 'PrA', 'CeA']
13
['ZrB', 'HfB', 'SnB', 'CeB', 'TiB']


In [379]:
# Calculate euclidean distance for desired atomic parameters
desired_params = [[RA_best, RB_best, MA_best, MB_best, Entropy_config_best]]

desired_params_norm = scaler.transform(desired_params)
print(desired_params_norm)

[[0.60659116 0.43396226 0.9951816  0.49841361 0.27463241]]


In [380]:
suggested_comps = []
best_comp = []
best_params = []

for i in tqdm(range(iter_i)):
    new_composition, num_A, num_B = gen_comp(num_A_lim, num_B_lim, data_species_A_c, data_species_B_c)

    dummy_array = np.zeros(len(data_species))

    # Calculate Crystallographic Params for new composition
    dummy_RA = 0
    dummy_RB = 0
    dummy_MA = 0
    dummy_MB = 0
    Entropy_A = 0
    Entropy_B = 0

    for key, val in new_composition.items():
        if key[-1] == 'A':
            atom_prop = get_atomic_property(key[:-1], 3, 'VIII')
            dummy_RA += val*atom_prop[0]
            dummy_MA += val*atom_prop[1]
            Entropy_A += 2*val*math.log(val)
        elif key[-1] == 'B':
            atom_prop = get_atomic_property(key[:-1], 4, 'VI')
            dummy_RB += val*atom_prop[0]
            dummy_MB += val*atom_prop[1]
            Entropy_B += 2*val*math.log(val)
        else:
            print('Error: A or B position not specified')

        dummy_entropy = (-(Entropy_A + Entropy_B - 0.11684 - 0.26)/11)*5
    
    
    dummy_params = [[dummy_RA, dummy_RB, dummy_MA, dummy_MB, dummy_entropy]]
    dummy_params_norm = scaler.transform(dummy_params)
    Distance_f = abstract_distance(desired_params_norm, dummy_params_norm)
    
    if Distance_f<=tol:
        if len(suggested_comps)>=100:
            for j, dist in enumerate(suggested_comps):
                if Distance_f<dist['Distance']:
                    new_composition.update({'Distance': Distance_f})
                    new_composition.update({'num_A': num_A})
                    new_composition.update({'num_B': num_B})
                    suggested_comps[j] = new_composition
                    break
                

        else:
            new_composition.update({'Distance': Distance_f})
            new_composition.update({'num_A': num_A})
            new_composition.update({'num_B': num_B})
            suggested_comps.append(new_composition)
            
        if Distance_f<best_distance:
            best_distance = Distance_f
            best_comp = new_composition
            best_params = dummy_params_norm

100%|████████████████████████████████████████████████████████████████████████| 500000/500000 [05:50<00:00, 1425.63it/s]


In [381]:
# Sort compositions by distance
#sorted_suggested_comp = dict(sorted(suggested_comps.items(), key=lambda item: item[1]['Distance']))
sorted_suggested_comp = sorted(suggested_comps, key=lambda item: item['Distance'])

In [382]:
print(suggested_comps)

[]


In [383]:
# Store top 10 compositions
n_comps = sorted_suggested_comp[:10]

n_comps_key = []
n_comps_val = []

for i, val in enumerate(n_comps):
    n_comps_key.append(i)
    n_comps_val.append(val)
    print(f'{i}: {val}')
    
n_comps_df = pd.DataFrame(n_comps_val, n_comps_key)

In [384]:
# Prep data for writing to csv
## Best composition and predicted parameters of composition
best_comp_key = []
best_comp_val = []

if not best_comp:
    best_comp_key = ['NaN']
    best_comp_val = ['NaN']
    print('Matching composition not found')
else:
    for key, val in best_comp.items():
        print(f'{key}: {val}')
        best_comp_key.append(key)
        best_comp_val.append(val)

    
best_params_key = []
best_params_val = []

if not best_comp:
    best_params_key = ['NaN']
    best_params_val = ['NaN']
else:
    for i, params in enumerate(best_params[0]):
        params_u = float(unnormalize(params, min_values[i], max_values[i]))
        best_params_key.append(columns[i])
        best_params_val.append(params_u)
        print(f'{columns[i]}: {params_u}')
    
    best_params_key.append('Distance')
    best_params_val.append(best_distance)
    print(f'Distance: {best_distance}')

## Normalization parameters

norm_keys = []
norm_vals = []

for key, val in normalization_vals.items():
    if not key:
        key = 'None'
    norm_keys.append(key)
    norm_vals.append(val)
    
norm_df = pd.DataFrame(norm_vals, norm_keys)

## Chem species constraints

Chem_c_keys = ['num_A_lim', 'num_B_lim']
Chem_c_vals = [num_A_lim, num_B_lim]

for key, val in Chem_sp_C.items():
    if not key:
        key = 'None'
    Chem_c_keys.append(key)
    Chem_c_vals.append(val)
    
Chem_c_df = pd.DataFrame(Chem_c_vals, Chem_c_keys)

Matching composition not found


In [385]:
#Save Composition
d = date.today()
now = datetime.now()
t = int((now - now.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds())
file_name = f'Generated_composition_{d}_{t}.csv'

Composition = pd.DataFrame(best_comp_val, best_comp_key)
Params = pd.DataFrame(best_params_val, best_params_key)
Target_params = pd.DataFrame(desired_params[0], columns)

Head1 = pd.DataFrame([data_file],['training_data'])
Head1.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Head2 = pd.DataFrame([iter_i],['Iterations'])
Head2.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Composition.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Params.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Head3 = pd.DataFrame(['Target parameters'])
Head3.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Target_params.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Head4 = pd.DataFrame(['Constraints'])
Head4.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Chem_c_df.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
Head6 = pd.DataFrame(['Normalization parameters'])
Head6.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
norm_df.to_csv(results_dir + '\\' + file_name, mode='a', header=False)

Head7 = pd.DataFrame(['Top 10 compositions'])
Head7.to_csv(results_dir + '\\' + file_name, mode='a', header=False)
n_comps_df.to_csv(results_dir + '\\' + file_name, mode='a', header=True)